In [ ]:
#importing all of our libraries and data
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from tensorflow.keras.applications import VGG16, InceptionResNetV2, ResNet50, Xception

In [ ]:
#passing the path of the directory containing training and testing data
train_dir = "../input/fer2013/train"
test_dir = "../input/fer2013/test"

In [ ]:
IMG_SIZE = 48 #defining the size of our image

In [ ]:
#performing data augmentation on our data
train_datagen = image.ImageDataGenerator(#rotation_range = 180,
                                         width_shift_range = 0.1,
                                         height_shift_range = 0.1,
                                         #brightness_range = [0.1,1.1],
                                         horizontal_flip = True,
                                         #vertical_flip = True,
                                         rescale = 1./255,
                                         zoom_range = 0.2,
                                         validation_split = 0.2
                                        )
validation_datagen = image.ImageDataGenerator(rescale = 1./255,
                                             validation_split = 0.2)

In [ ]:
#taking the path of the directory and generating batches of our augmented data
train_generator = train_datagen.flow_from_directory(directory = train_dir,
                                                               target_size = (IMG_SIZE,IMG_SIZE),
                                                               color_mode = "grayscale",
                                                               class_mode = "categorical",
                                                               batch_size = 64,
                                                               #shuffle = False,
                                                               subset = "training"
)

validation_generator = validation_datagen.flow_from_directory(directory = test_dir,
                                                             target_size = (IMG_SIZE,IMG_SIZE),
                                                             color_mode = "grayscale",
                                                             class_mode = "categorical",
                                                             batch_size = 64,
                                                             #shuffle = True,
                                                             subset = "validation"
)

In [ ]:
#model = tf.keras.Sequential()
#model.add(VGG16(include_top = False,weights = '../input/keras-pretrained-models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',input_shape= (IMG_SIZE,IMG_SIZE,3)))
#model.add(MaxPool2D(padding = 'same'))
#model.add(BatchNormalization(axis=-1))
#model.add(Flatten())
#model.add(Dense(7,activation = 'softmax'))

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64,(3,3),activation = 'relu',input_shape = (IMG_SIZE,IMG_SIZE,1),padding = 'same'),
    tf.keras.layers.Conv2D(64,(3,3),activation = 'relu',padding = 'same'),
    tf.keras.layers.MaxPool2D(pool_size=2,strides=2),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Conv2D(128,(3,3),activation= 'relu',padding = 'same'),
    tf.keras.layers.Conv2D(128,(3,3),activation= 'relu',padding = 'same'),
    tf.keras.layers.MaxPool2D(pool_size=2,strides=2),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Conv2D(256,(3,3),activation= 'relu',padding = 'same'),
    tf.keras.layers.Conv2D(256,(3,3),activation= 'relu',padding = 'same'),
    tf.keras.layers.Conv2D(256,(3,3),activation= 'relu',padding = 'same'),
    tf.keras.layers.MaxPool2D(pool_size=2,strides=2),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Conv2D(512,(3,3),activation= 'relu',padding = 'same'),
    tf.keras.layers.Conv2D(512,(3,3),activation= 'relu',padding = 'same'),
    tf.keras.layers.Conv2D(512,(3,3),activation= 'relu',padding = 'same'),
    tf.keras.layers.MaxPool2D(pool_size=2,strides=2),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Conv2D(512,(3,3),activation= 'relu',padding = 'same'),
    tf.keras.layers.Conv2D(512,(3,3),activation= 'relu',padding = 'same'),
    tf.keras.layers.Conv2D(512,(3,3),activation= 'relu',padding = 'same'),
    tf.keras.layers.MaxPool2D(pool_size=2,strides=2),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4096,activation = 'relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4096,activation = 'relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(7,activation = 'softmax')
])

In [ ]:
from tensorflow.keras.optimizers import Adam,RMSprop,SGD,Adamax
optimizer = Adam(lr = 0.0001)
model.compile(loss = "categorical_crossentropy",optimizer = optimizer,metrics = ['accuracy'])

In [ ]:
epochs = 30
batch_size = 64

In [ ]:
model.summary()

In [ ]:
history = model.fit(train_generator, steps_per_epoch =22968//64, epochs = epochs,validation_data = validation_generator,validation_steps = 1432//64)